In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5854,2020-11-28T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1519,247,1766,38116,...,799,1189,20181,1461,37424.0,24100.0,61524,942356,644441.0,NaN
5855,2020-11-28T17:00:00,ITA,9,Toscana,43.769231,11.255889,1593,279,1872,43200,...,-1459,1196,54005,2563,77917.0,23723.0,101640,1543449,948551.0,NaN
5856,2020-11-28T17:00:00,ITA,10,Umbria,43.106758,12.388247,351,64,415,8238,...,-397,348,14388,389,6545.0,16885.0,23430,412818,223404.0,NaN
5857,2020-11-28T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,130,14,144,1321,...,-82,72,4641,310,5714.0,702.0,6416,59088,33335.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5854,2020-11-28T17:00:00,19,Sicilia,1519,247,1766,38116,39882,799,1189,20181,1461,37424.0,24100.0,61524,942356,644441.0
5855,2020-11-28T17:00:00,9,Toscana,1593,279,1872,43200,45072,-1459,1196,54005,2563,77917.0,23723.0,101640,1543449,948551.0
5856,2020-11-28T17:00:00,10,Umbria,351,64,415,8238,8653,-397,348,14388,389,6545.0,16885.0,23430,412818,223404.0
5857,2020-11-28T17:00:00,2,Valle d'Aosta,130,14,144,1321,1465,-82,72,4641,310,5714.0,702.0,6416,59088,33335.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-28', '2020-11-27')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-28T17:00:00,1,Piemonte,4781,384,5165,70912,76077,-1512,2157,82163,6070,69065.0,95245.0,164310,1532290,950345.0
1,2020-11-28T17:00:00,2,Valle d'Aosta,130,14,144,1321,1465,-82,72,4641,310,5714.0,702.0,6416,59088,33335.0
2,2020-11-28T17:00:00,3,Lombardia,7616,919,8535,121780,130315,-240,4615,250832,21512,296506.0,106153.0,402659,4027257,2333345.0
3,2020-11-28T17:00:00,5,Veneto,2654,309,2963,75609,78572,1684,3498,58756,3644,36159.0,104813.0,140972,2769213,1055933.0
4,2020-11-28T17:00:00,6,Friuli Venezia Giulia,604,53,657,13339,13996,-259,1432,14598,801,25140.0,4255.0,29395,709897,278729.0
5,2020-11-28T17:00:00,7,Liguria,968,117,1085,11640,12725,-286,454,35925,2361,34709.0,16302.0,51011,594426,288865.0
6,2020-11-28T17:00:00,8,Emilia-Romagna,2673,245,2918,66865,69783,-506,2172,43717,5684,73837.0,45347.0,119184,2096063,1068411.0
7,2020-11-28T17:00:00,9,Toscana,1593,279,1872,43200,45072,-1459,1196,54005,2563,77917.0,23723.0,101640,1543449,948551.0
8,2020-11-28T17:00:00,10,Umbria,351,64,415,8238,8653,-397,348,14388,389,6545.0,16885.0,23430,412818,223404.0
9,2020-11-28T17:00:00,11,Marche,571,85,656,16587,17243,400,473,10551,1242,22435.0,6601.0,29036,406233,236940.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-27T17:00:00,1,Piemonte,4922,395,5317,72272,77589,-817,3149,78592,5972,68443.0,93710.0,162153,1513856,942459.0
1,2020-11-27T17:00:00,2,Valle d'Aosta,129,15,144,1403,1547,-66,101,4495,302,5651.0,693.0,6344,58339,33067.0
2,2020-11-27T17:00:00,3,Lombardia,7869,925,8794,121761,130555,-9846,5389,246096,21393,293815.0,104229.0,398044,3989971,2319630.0
3,2020-11-27T17:00:00,5,Veneto,2440,303,2743,74145,76888,1736,3418,57025,3561,35556.0,101918.0,137474,2750906,1050631.0
4,2020-11-27T17:00:00,6,Friuli Venezia Giulia,594,55,649,13606,14255,433,864,12932,776,23771.0,4192.0,27963,694896,272700.0
5,2020-11-27T17:00:00,7,Liguria,1019,109,1128,11883,13011,-481,606,35210,2336,34482.0,16075.0,50557,589686,287369.0
6,2020-11-27T17:00:00,8,Emilia-Romagna,2673,250,2923,67366,70289,-1183,2165,41116,5614,72676.0,44343.0,117019,2078822,1061899.0
7,2020-11-27T17:00:00,9,Toscana,1647,278,1925,44606,46531,-1604,1117,51397,2516,77248.0,23196.0,100444,1527078,942239.0
8,2020-11-27T17:00:00,10,Umbria,353,62,415,8635,9050,-384,261,13649,383,6463.0,16619.0,23082,408556,222157.0
9,2020-11-27T17:00:00,11,Marche,564,91,655,16188,16843,434,490,10490,1230,22062.0,6501.0,28563,403262,235330.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-28T17:00:00,1,Piemonte,4781,384,5165,70912,76077,-1512,2157,82163,6070,69065.0,95245.0,164310,1532290,950345.0
1,2020-11-28T17:00:00,2,Valle d'Aosta,130,14,144,1321,1465,-82,72,4641,310,5714.0,702.0,6416,59088,33335.0
2,2020-11-28T17:00:00,3,Lombardia,7616,919,8535,121780,130315,-240,4615,250832,21512,296506.0,106153.0,402659,4027257,2333345.0
3,2020-11-28T17:00:00,4,Trentino-Alto Adige,813,78,891,12800,13691,-101,713,23606,1152,30899.0,7550.0,38449,678297,282089.0
4,2020-11-28T17:00:00,5,Veneto,2654,309,2963,75609,78572,1684,3498,58756,3644,36159.0,104813.0,140972,2769213,1055933.0
5,2020-11-28T17:00:00,6,Friuli Venezia Giulia,604,53,657,13339,13996,-259,1432,14598,801,25140.0,4255.0,29395,709897,278729.0
6,2020-11-28T17:00:00,7,Liguria,968,117,1085,11640,12725,-286,454,35925,2361,34709.0,16302.0,51011,594426,288865.0
7,2020-11-28T17:00:00,8,Emilia-Romagna,2673,245,2918,66865,69783,-506,2172,43717,5684,73837.0,45347.0,119184,2096063,1068411.0
8,2020-11-28T17:00:00,9,Toscana,1593,279,1872,43200,45072,-1459,1196,54005,2563,77917.0,23723.0,101640,1543449,948551.0
9,2020-11-28T17:00:00,10,Umbria,351,64,415,8238,8653,-397,348,14388,389,6545.0,16885.0,23430,412818,223404.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-27T17:00:00,1,Piemonte,4922,395,5317,72272,77589,-817,3149,78592,5972,68443.0,93710.0,162153,1513856,942459.0
1,1,2020-11-27T17:00:00,2,Valle d'Aosta,129,15,144,1403,1547,-66,101,4495,302,5651.0,693.0,6344,58339,33067.0
2,2,2020-11-27T17:00:00,3,Lombardia,7869,925,8794,121761,130555,-9846,5389,246096,21393,293815.0,104229.0,398044,3989971,2319630.0
3,19,2020-11-27T17:00:00,4,Trentino-Alto Adige,822,86,908,12884,13792,-20,639,22801,1143,30337.0,7399.0,37736,671955,279881.0
4,3,2020-11-27T17:00:00,5,Veneto,2440,303,2743,74145,76888,1736,3418,57025,3561,35556.0,101918.0,137474,2750906,1050631.0
5,4,2020-11-27T17:00:00,6,Friuli Venezia Giulia,594,55,649,13606,14255,433,864,12932,776,23771.0,4192.0,27963,694896,272700.0
6,5,2020-11-27T17:00:00,7,Liguria,1019,109,1128,11883,13011,-481,606,35210,2336,34482.0,16075.0,50557,589686,287369.0
7,6,2020-11-27T17:00:00,8,Emilia-Romagna,2673,250,2923,67366,70289,-1183,2165,41116,5614,72676.0,44343.0,117019,2078822,1061899.0
8,7,2020-11-27T17:00:00,9,Toscana,1647,278,1925,44606,46531,-1604,1117,51397,2516,77248.0,23196.0,100444,1527078,942239.0
9,8,2020-11-27T17:00:00,10,Umbria,353,62,415,8635,9050,-384,261,13649,383,6463.0,16619.0,23082,408556,222157.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-28T17:00:00,1,Piemonte,4781,384,5165,70912,76077,-1512,2157,82163,6070,69065.0,95245.0,164310,1532290,950345.0,4341375
1,2020-11-28T17:00:00,2,Valle d'Aosta,130,14,144,1321,1465,-82,72,4641,310,5714.0,702.0,6416,59088,33335.0,125501
2,2020-11-28T17:00:00,3,Lombardia,7616,919,8535,121780,130315,-240,4615,250832,21512,296506.0,106153.0,402659,4027257,2333345.0,10103969
3,2020-11-28T17:00:00,4,Trentino-Alto Adige,813,78,891,12800,13691,-101,713,23606,1152,30899.0,7550.0,38449,678297,282089.0,1074819
4,2020-11-28T17:00:00,5,Veneto,2654,309,2963,75609,78572,1684,3498,58756,3644,36159.0,104813.0,140972,2769213,1055933.0,4907704
5,2020-11-28T17:00:00,6,Friuli Venezia Giulia,604,53,657,13339,13996,-259,1432,14598,801,25140.0,4255.0,29395,709897,278729.0,1211357
6,2020-11-28T17:00:00,7,Liguria,968,117,1085,11640,12725,-286,454,35925,2361,34709.0,16302.0,51011,594426,288865.0,1543127
7,2020-11-28T17:00:00,8,Emilia-Romagna,2673,245,2918,66865,69783,-506,2172,43717,5684,73837.0,45347.0,119184,2096063,1068411.0,4467118
8,2020-11-28T17:00:00,9,Toscana,1593,279,1872,43200,45072,-1459,1196,54005,2563,77917.0,23723.0,101640,1543449,948551.0,3722729
9,2020-11-28T17:00:00,10,Umbria,351,64,415,8238,8653,-397,348,14388,389,6545.0,16885.0,23430,412818,223404.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-28T17:00:00,1,Piemonte,4781,384,5165,70912,76077,-1512,2157,82163,6070,69065.0,95245.0,164310,1532290,950345.0,4341375,575
1,2020-11-28T17:00:00,2,Valle d'Aosta,130,14,144,1321,1465,-82,72,4641,310,5714.0,702.0,6416,59088,33335.0,125501,20
2,2020-11-28T17:00:00,3,Lombardia,7616,919,8535,121780,130315,-240,4615,250832,21512,296506.0,106153.0,402659,4027257,2333345.0,10103969,1036
3,2020-11-28T17:00:00,4,Trentino-Alto Adige,813,78,891,12800,13691,-101,713,23606,1152,30899.0,7550.0,38449,678297,282089.0,1074819,106
4,2020-11-28T17:00:00,5,Veneto,2654,309,2963,75609,78572,1684,3498,58756,3644,36159.0,104813.0,140972,2769213,1055933.0,4907704,1016
5,2020-11-28T17:00:00,6,Friuli Venezia Giulia,604,53,657,13339,13996,-259,1432,14598,801,25140.0,4255.0,29395,709897,278729.0,1211357,180
6,2020-11-28T17:00:00,7,Liguria,968,117,1085,11640,12725,-286,454,35925,2361,34709.0,16302.0,51011,594426,288865.0,1543127,209
7,2020-11-28T17:00:00,8,Emilia-Romagna,2673,245,2918,66865,69783,-506,2172,43717,5684,73837.0,45347.0,119184,2096063,1068411.0,4467118,634
8,2020-11-28T17:00:00,9,Toscana,1593,279,1872,43200,45072,-1459,1196,54005,2563,77917.0,23723.0,101640,1543449,948551.0,3722729,523
9,2020-11-28T17:00:00,10,Umbria,351,64,415,8238,8653,-397,348,14388,389,6545.0,16885.0,23430,412818,223404.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-28T17:00:00,3,Lombardia,7616,919,8535,121780,130315,-240,4615,...,88.71,19,-774,37286,13715.0,119,4736,12.38,0.04568,3.99
1,2020-11-28T17:00:00,5,Veneto,2654,309,2963,75609,78572,1684,3498,...,30.41,1464,80,18307,5302.0,83,1731,19.11,0.07128,2.87
2,2020-11-28T17:00:00,15,Campania,2164,186,2350,102033,104383,401,2729,...,36.83,423,-195,22007,14374.0,49,2279,12.40,0.04717,2.62
3,2020-11-28T17:00:00,8,Emilia-Romagna,2673,245,2918,66865,69783,-506,2172,...,38.64,-501,7,17241,6512.0,70,2601,12.60,0.04862,2.67
4,2020-11-28T17:00:00,1,Piemonte,4781,384,5165,70912,76077,-1512,2157,...,66.78,-1360,-992,18434,7886.0,98,3571,11.70,0.04968,3.78
5,2020-11-28T17:00:00,12,Lazio,3352,356,3708,85554,89262,1328,2070,...,42.03,1382,-206,28493,22951.0,25,717,7.26,0.03529,1.98
6,2020-11-28T17:00:00,16,Puglia,1675,200,1875,35779,37654,1237,1573,...,54.20,1217,-164,10032,6885.0,30,306,15.68,0.03924,1.31
7,2020-11-28T17:00:00,6,Friuli Venezia Giulia,604,53,657,13339,13996,-259,1432,...,29.44,-267,568,15001,6029.0,25,1666,9.55,0.11821,2.43
8,2020-11-28T17:00:00,9,Toscana,1593,279,1872,43200,45072,-1459,1196,...,53.35,-1406,79,16371,6312.0,47,2608,7.31,0.03213,2.73
9,2020-11-28T17:00:00,19,Sicilia,1519,247,1766,38116,39882,799,1189,...,42.01,822,-377,8777,5184.0,43,347,13.55,0.02393,1.24


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,919,117,1036,88.71
1,Trentino-Alto Adige,78,28,106,73.58
2,Valle d'Aosta,14,6,20,70.00
3,Piemonte,384,191,575,66.78
4,Marche,85,58,143,59.44
5,Liguria,117,92,209,55.98
6,Umbria,64,53,117,54.70
7,Puglia,200,169,369,54.20
8,Toscana,279,244,523,53.35
9,Abruzzo,77,71,148,52.03


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,2020-11-26T17:00:00,22,P.A. Trento,417,43,460,1902,2362,-85,297,11867,615,7579.0,7265.0,14844,363874,131463.0
554,2020-11-27T17:00:00,21,P.A. Bolzano,414,42,456,11027,11483,33,410,10660,520,22663.0,0.0,22663,304527,147068.0
555,2020-11-27T17:00:00,22,P.A. Trento,408,44,452,1857,2309,-53,229,12141,623,7674.0,7399.0,15073,367428,132813.0
556,2020-11-28T17:00:00,21,P.A. Bolzano,409,36,445,11113,11558,75,494,11076,523,23157.0,0.0,23157,306915,147600.0


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,2020-11-24T17:00:00,4,Trentino-Alto Adige,895,82,977,13151,14128,-5,303,...,30.0,3649.0,1169.0,5.0,8.39,6.92,8.30,25.92,1.23,-0.04
275,2020-11-25T17:00:00,4,Trentino-Alto Adige,863,82,945,13102,14047,-81,520,...,15.0,6135.0,1176.0,-49.0,8.68,6.73,8.48,44.22,0.00,-0.57
276,2020-11-26T17:00:00,4,Trentino-Alto Adige,851,83,934,12878,13812,-235,589,...,20.0,6425.0,1807.0,-224.0,8.89,6.76,9.17,32.60,1.22,-1.67
277,2020-11-27T17:00:00,4,Trentino-Alto Adige,822,86,908,12884,13792,-20,639,...,19.0,5855.0,1903.0,6.0,9.47,6.58,10.91,33.58,3.61,-0.14


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
